### Description:

- Reads sun glare dataset and visualizes it
- Also overlaps other metrics

In [ ]:
import folium


def get_circle_color_for_sun_glare(sun_glare_row):

    if sun_glare_row['angle_risk'] == True:
        if sun_glare_row['has_sun_glare'] == True:
            # sun glare not blocked
            return 'red'
        else:
            # sun glare blocked
            return 'yellow'
    # no chance of sun glare
    return 'green'



def visualize_just_data(sun_glare_dataset, segments_dataset, center_point):

    m = folium.Map(location=center_point, zoom_start=16)

    # TODO find a way to draw lines between the points that are going in the same heading
    # ex: 2 way streets will have 2 lines, one for each direction

    # TODO maybe request lineString from the osnmx edges, then use that to draw the lines (instead of straight lines) 

    #        Yellow       Red
    #    ----node---- ----node----   -->
    #        Green        Green
    #    ----node---- ----node----   <--
    # half the ditance between the two nodes is colored one color, the other half is colored another color

    row_counter = 0
    for _, row in sun_glare_dataset.iterrows():

        
        
        row_counter += 1
        circle_color = get_circle_color_for_sun_glare(row)

        lat = row['lat']
        long = row['long']

        folium.CircleMarker(
            location=[lat, long],
            radius=5,
            color=circle_color,
            fill=True,
            fill_opacity=0.6
        ).add_to(m)

    m.save(f"saved_sun_glare_map.html")
    print(f"Map created with {row_counter} detections.")




In [ ]:
import pandas as pd

def main():
    sun_glare_dataset_path = "data/sun_glare_dict.csv"
    segments_dataset_path = "data/dc_segments.csv"

    sun_glare_data = pd.read_csv(sun_glare_dataset_path)
    segments_data = pd.read_csv(segments_dataset_path)

    center_point = (38.89565719232077, -77.04168192501736)

    visualize_just_data(sun_glare_data, segments_data, center_point)


main()